# Ontology-guided Radiomics Analysis Workflow (O-RAW) verison 2.2
We propose an open-source Ontology-guided Radiomics Analysis Workflow (O-RAW) to address the above challenges in the following manner: (i) distributing a free and open-source software package for radiomics analysis, (ii) deploying a standard lexicon to uniquely describe features in common usage and (iii) provide methods to publish radiomic features as a semantically-interoperable data graph object complying to FAIR (Findable Accessible Interoperable Reusable) data principles. 

## This repository accompanies the publication 'Technical Note: Ontology-guided Radiomics Analysis Workflow (O-RAW)'
**If you publish any work which uses this package, please cite the following publication:**
*Shi, Zhenwei, Alberto Traverso, Johan van Soest, Andre Dekker, and Leonard Wee. "Ontology‐guided Radiomics Analysis Workflow (O‐RAW)." Medical Physics (2019). https://aapm.onlinelibrary.wiley.com/doi/pdf/10.1002/mp.13844*

In [5]:
'''
###############################
@author: zhenwei.shi, Maastro##
###############################

Usage for a given case: python HelloORAW.py 

Read parameters:

roi = 'all' (calculate radiomics on all ROIs)
roi = '[Gg][Tt][Vv]' (calculate radiomics on a specific ROI or a type of ROI)
export_format = 'csv' OR 'rdf'
export_name = 'ORAW_test'
walk_dir = './data/CT' (directory of your data)
######################################
'''

"\n###############################\n@author: zhenwei.shi, Maastro##\n###############################\n\nUsage for a given case: python HelloORAW.py \n\nRead parameters:\n\nroi = 'all' (calculate radiomics on all ROIs)\nroi = '[Gg][Tt][Vv]' (calculate radiomics on a specific ROI or a type of ROI)\nexport_format = 'csv' OR 'rdf'\nexport_name = 'ORAW_test'\nwalk_dir = './data/CT' (directory of your data)\n######################################\n"

In [9]:
import sys
sys.path.append('../')

In [15]:
from __future__ import print_function

from time import process_time
import os,yaml
import ORAW
import glob
import shutil
import pandas as pd
from DicomDatabase import DicomDatabase

In [16]:
#-------------------------USER-------------------------------
#----------------O-RAW initial parameters -------------------
# roi = 'all'
roi = '[Gg][Tt][Vv]'  
export_format = 'csv'
# export_format = 'rdf'
export_name = 'ORAW_'
walk_dir = '../data/CT'

In [17]:
#-----------------create tmp CT/STRUCT directories-----------
CTWorkingDir = "../CTFolder"
STRUCTWorkingDir = "../StructFolder"

if not os.path.exists(CTWorkingDir):
  os.makedirs(CTWorkingDir)
if not os.path.exists(STRUCTWorkingDir):
  os.makedirs(STRUCTWorkingDir)

# -----------------------------------------------------------
# initialize dicom DB
dicomDb = DicomDatabase()
# walk over all files in folder, and index in the database
dicomDb.parseFolder(walk_dir)

excludeStructRegex = "(Patient.*|BODY.*|Body.*|NS.*|Couch.*)"
if os.environ.get("EXCLUDE_STRUCTURE_REGEX") is not None:
    excludeStructRegex = os.environ.get("EXCLUDE_STRUCTURE_REGEX")
# ----------------------------------------------------
if export_format == 'rdf':
    exportDir = '../RFstore/Turtle_output' # export format is RDF
else:
    exportDir = '../RFstore/CSV_output' # export format is CSV
# loop over patients
for ptid in dicomDb.getPatientIds():
    print("staring with Patient %s" % (ptid))
    # get patient by ID
    myPatient = dicomDb.getPatient(ptid)
    # loop over RTStructs of this patient
    for myStructUID in myPatient.getRTStructs():
        print("Starting with RTStruct %s" % myStructUID)
        # Get RTSTRUCT by SOP Instance UID
        myStruct = myPatient.getRTStruct(myStructUID)
        # Get CT which is referenced by this RTStruct, and is linked to the same patient
        # mind that this can be None, as only a struct, without corresponding CT scan is found
        myCT = myPatient.getCTForRTStruct(myStruct)

        # check if the temperal CT/STRUCT folder is empty
        if not (os.listdir(CTWorkingDir)==[] and os.listdir(STRUCTWorkingDir)==[]):
          ct_files = glob.glob(os.path.join(CTWorkingDir,'*'))
          for f in ct_files:
            os.remove(f)

          struct_files = glob.glob(os.path.join(STRUCTWorkingDir,'*'))
          for f in struct_files:
            os.remove(f)
        
        #only show if we have both RTStruct and CT
        if myCT is not None:
            # copy RTSTRUCT file to tmp folder as 'struct.dcm'
            shutil.copy2(myStruct.getFileLocation(),os.path.join(STRUCTWorkingDir,'struct.dcm'))
            # copy DICOM slices to tmp folder as 'struct.dcm'
            slices = myCT.getSlices()
            for i in range(len(slices)):
                shutil.copy2(slices[i],os.path.join(CTWorkingDir,str(i)+".dcm"))
            #graph = ORAW_Docker.executeORAWbatch_all([ptid],roi,myStructUID,exportDir,export_format,export_name,[CTWorkingDir],[STRUCTWorkingDir],excludeStructRegex)
            if roi == 'all':
                ORAW.executeORAWbatch_all([ptid],roi,myStructUID,exportDir,export_format,export_name,[CTWorkingDir],[STRUCTWorkingDir],excludeStructRegex)
            else:
                ORAW.executeORAWbatch_roi([ptid],roi,myStructUID,exportDir,export_format,export_name,[CTWorkingDir],[STRUCTWorkingDir],excludeStructRegex)
            #####################
        print("Done for struct %s of patient %s" % (myStructUID, ptid))

staring with Patient LUNG1-002
Starting with RTStruct 1.3.6.1.4.1.40744.29.58200769026013337550235117523425780982
Generating binary mask based on ROI: GTV-1 ......


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


LUNG1-002 GTV-1
Done for struct 1.3.6.1.4.1.40744.29.58200769026013337550235117523425780982 of patient LUNG1-002
